# 🎓 Multi-Level Chunking: Complete Interactive Tutorial

Welcome! This notebook will help you understand **multi-level chunking** - a powerful technique for building better RAG systems.

## 📚 What You'll Learn

1. What is chunking and why it matters
2. Problems with traditional single-level chunking
3. How multi-level chunking works
4. Step-by-step search process with real examples
5. Hands-on exercises to test your understanding

## 📋 Table of Contents
1. [Introduction](#introduction)
2. [The Problem with Traditional Chunking](#problem)
3. [Understanding Multi-Level Chunking](#understanding)
4. [Practical Example](#example)
5. [How Search Works](#search)
6. [Complete Query Flow](#flow)
7. [Interactive Exercises](#exercises)
8. [Summary](#summary)

---
**Time to complete**: 30-45 minutes

**Prerequisites**: Basic understanding of RAG systems

## 1. Introduction <a name="introduction"></a>

### What is Chunking?

**Chunking** is the process of breaking down large documents into smaller pieces for information retrieval.

### Why Do We Need It?

```
Problem: A 10,000-word medical paper
   ↓
LLM can only process ~8,000 tokens at once
   ↓
Solution: Break it into manageable chunks
   ↓
Search only relevant chunks
   ↓
Give LLM only what it needs
```

### Real-World Analogy

Think of a **medical textbook**:

- 📚 **Book** (Level 1): Overview of all topics
- 📖 **Chapters** (Level 2): Major disease categories  
- 📄 **Sections** (Level 3): Specific diseases
- 📝 **Paragraphs** (Level 4): Detailed symptoms, treatments, statistics

When someone asks "What's the treatment for pneumonia?", you don't read the entire book - you go straight to the relevant paragraph!

---
## 2. The Problem with Traditional Chunking <a name="problem"></a>

### Traditional Approach

Most RAG systems use **fixed-size chunks**:

```
Document (10,000 words)
    ↓
Split every 500 words
    ↓
Chunk 1 | Chunk 2 | Chunk 3 | ... | Chunk 20
```

### Problems:

| Problem | Example |
|---------|----------|
| **Lost Context** | "The p-value was" [CHUNK 1] <br> "<0.001 indicating..." [CHUNK 2] |
| **Fixed Size** | Important paragraph = Same size as filler text |
| **No Hierarchy** | Can't tell if chunk is intro or conclusion |
| **Poor Retrieval** | Hard to get both overview AND details |

Let's see this in action:

In [ ]:
# Traditional Chunking Example

sample_doc = """
Title: Antifungal Therapies for ABPA in Cystic Fibrosis

Abstract Background: Allergic bronchopulmonary aspergillosis (ABPA) is a common 
complication in cystic fibrosis patients causing significant morbidity.

Objectives: To assess the efficacy of antifungal therapies for ABPA.

Methods: Randomized controlled trial. Participants: 45 cystic fibrosis patients 
with ABPA. Intervention: Itraconazole 200mg twice daily for 16 weeks.

Results: Mean FEV1% predicted improved by 12.3% (95% CI: 8.7-15.9, p<0.001) 
in the itraconazole group. Secondary outcomes: 40% reduction in exacerbation 
frequency (p=0.02). Adverse events: Nausea (23%), vomiting (15%), elevated 
liver enzymes (8%).
"""

def traditional_chunk(text, words_per_chunk=30):
    words = text.split()
    return [' '.join(words[i:i+words_per_chunk]) 
            for i in range(0, len(words), words_per_chunk)]

chunks = traditional_chunk(sample_doc)

print(f"Created {len(chunks)} chunks\n")
for i, chunk in enumerate(chunks, 1):
    print(f"\n{'='*70}")
    print(f"CHUNK {i}")
    print('='*70)
    print(chunk)

print("\n⚠️  PROBLEMS:")
print("  1. Statistical data split across chunks")
print("  2. No context about what section each chunk belongs to")
print("  3. Can't tell which chunks are more important")

---
## 3. Understanding Multi-Level Chunking <a name="understanding"></a>

### The Solution: Hierarchical Chunking

Instead of fixed-size chunks, create a **hierarchy**:

```
Level 1: DOCUMENT
├── Level 2: ABSTRACT SECTION
│   ├── Level 3: Background Subsection
│   │   └── Level 4: "ABPA is a common complication..."
│   ├── Level 3: Objectives Subsection
│   │   └── Level 4: "To assess efficacy..."
│   └── Level 3: Methods Subsection
│       └── Level 4: "Randomized controlled trial..."
├── Level 2: RESULTS SECTION
│   ├── Level 3: Primary Outcomes
│   │   └── Level 4: "FEV1 improved 12.3% (p<0.001)"
│   └── Level 3: Adverse Events
│       └── Level 4: "Nausea (23%), vomiting (15%)..."
└── Level 2: DISCUSSION SECTION
```

### The Four Levels

| Level | Name | Content | Weight | When to Use |
|-------|------|---------|--------|-------------|
| **1** | Document | Title, authors, summary | 0.1 | "Tell me about..." |
| **2** | Section | Complete sections | 0.2 | "What methods?" |
| **3** | Subsection | Detailed components | 0.4 | "What were primary outcomes?" |
| **4** | Paragraph | Specific statements | 0.6-0.7 | "What was the p-value?" |

### Key Insight 💡

**Weights indicate importance for different query types:**
- Higher weight (0.6-0.7) = More important for **specific** queries
- Lower weight (0.1) = More important for **broad** queries

This is the **magic** of multi-level chunking!

### Visual Representation

```
┌─────────────────────────────────────────────────────────────┐
│ LEVEL 1: Document Chunk (Weight: 0.1)                      │
│ "Antifungal Therapies for ABPA - Smith et al. - Quality A" │
│ USE: Broad queries like "Tell me about ABPA treatment"      │
└─────────────────────────────────────────────────────────────┘
        │
        ├──────────────────┬──────────────────┐
        ↓                  ↓                  ↓
┌──────────────────┐ ┌──────────────────┐ ┌──────────────────┐
│ LEVEL 2: Abstract│ │ LEVEL 2: Methods │ │ LEVEL 2: Results │
│ (Weight: 0.2)    │ │ (Weight: 0.2)    │ │ (Weight: 0.2)    │
│ Complete section │ │ Complete section │ │ Complete section │
└──────────────────┘ └──────────────────┘ └──────────────────┘
        │                                          │
        ├────────────┐                             ├────────────┐
        ↓            ↓                             ↓            ↓
┌──────────────┐ ┌──────────────┐      ┌──────────────┐ ┌──────────────┐
│ LEVEL 3:     │ │ LEVEL 3:     │      │ LEVEL 3:     │ │ LEVEL 3:     │
│ Background   │ │ Objectives   │      │ Primary      │ │ Adverse      │
│ (Weight: 0.4)│ │ (Weight: 0.4)│      │ Outcomes     │ │ Events       │
│              │ │              │      │ (Weight: 0.4)│ │ (Weight: 0.4)│
└──────────────┘ └──────────────┘      └──────────────┘ └──────────────┘
        │                                      │
        ↓                                      ↓
┌──────────────┐                      ┌──────────────┐
│ LEVEL 4:     │                      │ LEVEL 4:     │
│ "ABPA is a   │                      │ "FEV1        │
│ common..."   │                      │ improved     │
│ (Weight: 0.6)│                      │ 12.3%..."    │
│              │                      │ (Weight: 0.7)│
└──────────────┘                      └──────────────┘
```

---
## 4. Practical Example: Multi-Level Chunking <a name="example"></a>

Let's implement multi-level chunking for our sample document:

In [ ]:
class MultiLevelChunker:
    """Demonstrates multi-level chunking"""
    
    def __init__(self):
        self.chunks = {'level_1': [], 'level_2': [], 'level_3': [], 'level_4': []}
    
    def create_level_1(self):
        """Document-level chunk"""
        return {
            'level': 1,
            'content': 'Antifungal Therapies for ABPA in Cystic Fibrosis',
            'metadata': {'quality': 'A', 'authors': 'Smith et al.', 'topic': 'ABPA'},
            'weight': 0.1,
            'use_case': 'Broad search, overview'
        }
    
    def create_level_2(self):
        """Section-level chunks"""
        return [
            {'level': 2, 'section': 'Abstract', 'content': 'Background + Objectives + Methods summary', 'weight': 0.2},
            {'level': 2, 'section': 'Methods', 'content': 'RCT with 45 CF patients, itraconazole 16 weeks', 'weight': 0.2},
            {'level': 2, 'section': 'Results', 'content': 'Primary + secondary outcomes + adverse events', 'weight': 0.2}
        ]
    
    def create_level_3(self):
        """Subsection-level chunks"""
        return [
            {'level': 3, 'section': 'Abstract', 'subsection': 'Background', 
             'content': 'ABPA is a common complication in CF patients', 'weight': 0.4},
            {'level': 3, 'section': 'Results', 'subsection': 'Primary Outcomes',
             'content': 'Mean FEV1% improved by 12.3% (95% CI: 8.7-15.9, p<0.001)', 'weight': 0.4},
            {'level': 3, 'section': 'Results', 'subsection': 'Adverse Events',
             'content': 'Nausea (23%), vomiting (15%), elevated liver enzymes (8%)', 'weight': 0.4}
        ]
    
    def create_level_4(self):
        """Paragraph-level chunks"""
        return [
            {'level': 4, 'subsection': 'Primary Outcomes',
             'content': 'Mean FEV1% predicted improved by 12.3% (95% CI: 8.7-15.9, p<0.001)',
             'statistical_data': {'effect': '12.3%', 'ci': '8.7-15.9', 'p': '<0.001'},
             'weight': 0.7},
            {'level': 4, 'subsection': 'Adverse Events',
             'content': 'Nausea occurred in 23% of patients in itraconazole group',
             'weight': 0.6}
        ]
    
    def display_all(self):
        print("\n" + "="*80)
        print("MULTI-LEVEL CHUNKING STRUCTURE")
        print("="*80 + "\n")
        
        print("📄 LEVEL 1 (Weight: 0.1) - Document Overview")
        l1 = self.create_level_1()
        print(f"   Content: {l1['content']}")
        print(f"   Metadata: {l1['metadata']}")
        print(f"   Use: {l1['use_case']}\n")
        
        print("📑 LEVEL 2 (Weight: 0.2) - Sections")
        for chunk in self.create_level_2():
            print(f"   ├─ {chunk['section']}: {chunk['content']}")
        print()
        
        print("📋 LEVEL 3 (Weight: 0.4) - Subsections")
        for chunk in self.create_level_3():
            print(f"   ├─ {chunk['section']} → {chunk['subsection']}")
            print(f"   │  {chunk['content']}")
        print()
        
        print("📝 LEVEL 4 (Weight: 0.6-0.7) - Paragraphs")
        for chunk in self.create_level_4():
            print(f"   ├─ {chunk['subsection']}")
            print(f"   │  {chunk['content']}")
            if 'statistical_data' in chunk:
                print(f"   │  📊 Stats: {chunk['statistical_data']}")
        print()

# Run the example
chunker = MultiLevelChunker()
chunker.display_all()

print("\n✅ Notice how each level serves a different purpose!")
print("   Level 1: Quick overview")
print("   Level 2: Section summaries")
print("   Level 3: Detailed information")
print("   Level 4: Precise data with statistics")

---
## 5. How Search Works with Multi-Level Chunks <a name="search"></a>

### Important: Search is PARALLEL, not Sequential!

❌ **WRONG** (Sequential):
```
Search Level 1 → Get results → Then search Level 2 → Then Level 3 → Then Level 4
```

✅ **CORRECT** (Parallel):
```
Search ALL levels simultaneously → Combine results with weights → Rank by relevance
```

### The Search Process (6 Steps)

1. **Query Analysis**: Understand what user is asking
2. **Target Level Determination**: Decide which levels matter most
3. **Parallel Search**: Search all target levels at once
4. **Cross-Level Scoring**: Combine results with level weights
5. **Re-ranking**: Advanced scoring for top results
6. **Context Assembly**: Build hierarchical response

Let's implement this:

In [ ]:
class MultiLevelSearch:
    """Simulates multi-level search"""
    
    def __init__(self, chunker):
        self.chunker = chunker
        self.all_chunks = {
            1: [chunker.create_level_1()],
            2: chunker.create_level_2(),
            3: chunker.create_level_3(),
            4: chunker.create_level_4()
        }
    
    def analyze_query(self, query):
        """Step 1: Query Analysis"""
        print("\n" + "="*80)
        print("STEP 1: QUERY ANALYSIS")
        print("="*80)
        print(f"Query: \"{query}\"\n")
        
        q_lower = query.lower()
        
        # Determine intent and target levels
        if any(w in q_lower for w in ['p-value', 'percentage', 'ci', 'statistical']):
            intent, levels = 'statistical', [3, 4]
        elif any(w in q_lower for w in ['efficacy', 'outcome', 'side effect', 'adverse']):
            intent, levels = 'specific', [3, 4]
        elif any(w in q_lower for w in ['tell me', 'overview', 'about']):
            intent, levels = 'broad', [1, 2]
        else:
            intent, levels = 'general', [1, 2, 3, 4]
        
        print(f"📊 Intent: {intent}")
        print(f"🎯 Target Levels: {levels}")
        print(f"💡 Reason: {'Needs precise data' if intent in ['statistical', 'specific'] else 'Needs overview'}")
        
        return {'intent': intent, 'levels': levels}
    
    def parallel_search(self, query, target_levels):
        """Step 2: Parallel Multi-Level Search"""
        print("\n" + "="*80)
        print("STEP 2: PARALLEL SEARCH (All levels searched simultaneously!)")
        print("="*80 + "\n")
        
        keywords = [w for w in query.lower().split() if len(w) > 3]
        print(f"🔍 Keywords: {keywords}\n")
        
        all_results = []
        
        for level in target_levels:
            print(f"\nSearching Level {level}...")
            results = []
            
            for chunk in self.all_chunks[level]:
                content = str(chunk.get('content', '')).lower()
                score = sum(1 for kw in keywords if kw in content)
                
                if score > 0:
                    weighted_score = score * chunk['weight']
                    results.append({
                        'chunk': chunk,
                        'score': score,
                        'weighted_score': weighted_score,
                        'level': level
                    })
            
            print(f"  Found {len(results)} matches")
            for r in results[:2]:
                print(f"    Score: {r['score']}, Weighted: {r['weighted_score']:.2f}")
            
            all_results.extend(results)
        
        return sorted(all_results, key=lambda x: x['weighted_score'], reverse=True)
    
    def rerank(self, results, top_k=5):
        """Step 3: Re-ranking"""
        print("\n" + "="*80)
        print("STEP 3: RE-RANKING TOP RESULTS")
        print("="*80 + "\n")
        
        top = results[:top_k]
        
        for i, r in enumerate(top, 1):
            stat_bonus = 0.3 if 'statistical_data' in r['chunk'] else 0
            r['final_score'] = r['weighted_score'] + stat_bonus
            
            print(f"{i}. Level {r['level']} (Initial: {r['weighted_score']:.2f}, "
                  f"Bonus: +{stat_bonus:.1f}, Final: {r['final_score']:.2f})")
            print(f"   {r['chunk']['content'][:70]}...")
            if 'statistical_data' in r['chunk']:
                print(f"   📊 {r['chunk']['statistical_data']}")
        
        return sorted(top, key=lambda x: x['final_score'], reverse=True)
    
    def assemble_context(self, results):
        """Step 4: Context Assembly"""
        print("\n" + "="*80)
        print("STEP 4: HIERARCHICAL CONTEXT ASSEMBLY")
        print("="*80 + "\n")
        
        groups = {}
        for r in results:
            level = r['level']
            if level not in groups:
                groups[level] = []
            groups[level].append(r)
        
        if 4 in groups:
            print("📝 PRIMARY (Level 4 - 60% weight):")
            for r in groups[4][:2]:
                print(f"   • {r['chunk']['content']}")
        
        if 3 in groups:
            print("\n📋 SUPPORTING (Level 3 - 40% weight):")
            for r in groups[3][:2]:
                print(f"   • {r['chunk']['content']}")
        
        if 2 in groups:
            print("\n📑 BACKGROUND (Level 2 - 20% weight):")
            for r in groups[2][:1]:
                print(f"   • Section: {r['chunk']['section']}")
        
        if 1 in groups:
            print("\n📄 METADATA (Level 1 - 10% weight):")
            for r in groups[1][:1]:
                print(f"   • {r['chunk']['content']}")
        
        print("\n✅ Context ready for LLM!")
    
    def search(self, query):
        """Complete search"""
        analysis = self.analyze_query(query)
        results = self.parallel_search(query, analysis['levels'])
        
        if not results:
            print("\n⚠️ No results found")
            return
        
        ranked = self.rerank(results)
        self.assemble_context(ranked)
        return ranked

# Create search engine
search = MultiLevelSearch(chunker)
print("\n✅ Search engine initialized. Ready for queries!")

---
## 6. Complete Query Flow Examples <a name="flow"></a>

Let's see how different types of queries are handled:

In [ ]:
# Example 1: Statistical Query (Needs Level 4)
print("\n" + "#"*80)
print("# EXAMPLE 1: STATISTICAL QUERY")
print("#"*80)

search.search("What was the FEV1 improvement with itraconazole?")

In [ ]:
# Example 2: Safety Query (Needs Levels 3-4)
print("\n" + "#"*80)
print("# EXAMPLE 2: SAFETY QUERY")
print("#"*80)

search.search("What are the side effects of itraconazole?")

In [ ]:
# Example 3: Broad Overview (Needs Levels 1-2)
print("\n" + "#"*80)
print("# EXAMPLE 3: BROAD OVERVIEW QUERY")
print("#"*80)

search.search("Tell me about ABPA treatment")

---
## 7. Interactive Exercises <a name="exercises"></a>

### Exercise 1: Query Classification

For each query, determine:
1. What is the intent?
2. Which levels should be searched?
3. Why?

Try these queries:

In [ ]:
# Exercise 1
exercise_queries = [
    "What is the p-value for the primary outcome?",
    "Tell me about this study",
    "What methods were used?",
    "How many patients had nausea?"
]

print("\n" + "="*80)
print("EXERCISE 1: Classify These Queries")
print("="*80 + "\n")

for q in exercise_queries:
    print(f"\nQuery: \"{q}\"")
    print("Your answer:")
    print("  Intent: ___________")
    print("  Target Levels: ___________")
    print("  Reasoning: ___________")
    print()

print("\n💡 Run search.analyze_query() on each to check your answers!")

### Exercise 2: Understanding Weights

**Question**: Why does Level 4 have weight 0.6-0.7 while Level 1 has weight 0.1?

**Think about:**
- What information is at each level?
- Which gives the most precise answers?
- Which gives the broadest overview?

In [ ]:
# Exercise 2: Compare the impact of weights

print("\n" + "="*80)
print("EXERCISE 2: Weight Impact")
print("="*80 + "\n")

print("Scenario: User asks 'What was the p-value?'\n")

print("Level 1 (Weight 0.1):")
print("  Content: 'Antifungal Therapies for ABPA'")
print("  Keyword matches: 0")
print("  Score: 0 × 0.1 = 0\n")

print("Level 4 (Weight 0.7):")
print("  Content: 'FEV1 improved by 12.3% (p<0.001)'")
print("  Keyword matches: 1 ('p-value' found)")
print("  Score: 1 × 0.7 = 0.7\n")

print("✅ Result: Level 4 wins! Higher weight + relevant content = Best match")
print("\n💡 The weight system ensures specific queries get specific answers!")

### Exercise 3: Context Assembly

**Question**: Why combine results from multiple levels instead of just using the top result?

In [ ]:
# Exercise 3
print("\n" + "="*80)
print("EXERCISE 3: Single-Level vs Multi-Level Response")
print("="*80 + "\n")

print("Query: 'What was the efficacy of itraconazole?'\n")

print("❌ SINGLE-LEVEL (Only Level 4):")
print("  'Mean FEV1% improved by 12.3% (95% CI: 8.7-15.9, p<0.001)'")
print("\n  Problems:")
print("    • What study is this from?")
print("    • Who were the participants?")
print("    • What was the study design?")
print("    • Is this study reliable?\n")

print("✅ MULTI-LEVEL (Combined):")
print("  Level 4: 'FEV1 improved 12.3% (95% CI: 8.7-15.9, p<0.001)'")
print("  Level 3: 'Primary outcome: lung function at 16 weeks'")
print("  Level 2: 'RCT with 45 CF patients'")
print("  Level 1: 'Quality A study by Smith et al.'")
print("\n  ✨ User gets complete, trustworthy answer with context!")

print("\n💡 Multi-level context = Better understanding + Higher trust")

---
## 8. Summary and Key Takeaways <a name="summary"></a>

### 🎯 Core Concepts

1. **Multi-Level Hierarchy**
   - Level 1: Document overview (0.1 weight)
   - Level 2: Major sections (0.2 weight)
   - Level 3: Subsections (0.4 weight)
   - Level 4: Paragraphs (0.6-0.7 weight)

2. **Different Levels, Different Purposes**
   - Broad queries → Levels 1-2
   - Specific queries → Levels 3-4
   - Statistical queries → Level 4

3. **Parallel Search**
   - ❌ NOT sequential (Level 1 → 2 → 3 → 4)
   - ✅ All levels searched simultaneously
   - Results combined with level-specific weights

4. **Hierarchical Context**
   - Primary (Level 4): Most specific
   - Supporting (Level 3): Detailed context
   - Background (Levels 1-2): Overview

### 🚀 Advantages

| Aspect | Traditional | Multi-Level |
|--------|-------------|-------------|
| Granularity | Fixed | Adaptive |
| Context | Lost | Preserved |
| Precision | Same for all | Level-specific |
| Flexibility | One-size-fits-all | Query-adaptive |
| Relationships | None | Hierarchical |

### ✅ When to Use Multi-Level Chunking

**Good for:**
- 📄 Structured documents (papers, reports, medical reviews)
- 🎯 When precision matters (medical, legal, technical)
- 🔍 Users need both overview AND details
- 📊 Documents with clear hierarchy

**Not ideal for:**
- 📱 Unstructured text (social media, blogs)
- 📝 Very short documents
- 🌐 No clear sections
- 🔎 Simple keyword search sufficient

### 💡 The Big Picture

Multi-level chunking is like a **zoom lens**:
- **Zoomed out** (Levels 1-2): See the big picture
- **Zoomed in** (Levels 3-4): See specific details
- **Adaptive**: Automatically adjusts to user needs

### 📚 What's Next?

1. **Implementation**: Build multi-level chunking for your documents
2. **Embedding**: Generate embeddings for each level
3. **Vector DB**: Store chunks with hierarchical relationships
4. **Retrieval**: Implement parallel search with re-ranking
5. **Testing**: Evaluate on different query types

### ✅ Checklist

- [ ] I understand the four levels and their purposes
- [ ] I know why different levels have different weights
- [ ] I understand that search is parallel, not sequential
- [ ] I know how query type determines target levels
- [ ] I understand hierarchical context assembly
- [ ] I can explain advantages over traditional chunking
- [ ] I know when to use multi-level chunking

**If you checked all boxes, congratulations! You understand multi-level chunking!** 🎉

---
## 📖 Additional Resources

### Documentation
- `Expert_RAG_System_Approach.md`: Overall RAG system design
- `RAG_System_Flowchart.md`: Complete system flowchart
- `Multi_Level_Chunking_Search_Flowchart.md`: Detailed search process

### Research Papers
- Dense Passage Retrieval (Karpukhin et al., 2020)
- BERT (Devlin et al., 2018)
- Retrieval-Augmented Generation (Lewis et al., 2020)

### Implementation Tools
- **LangChain**: Multi-vector retriever
- **LlamaIndex**: Hierarchical node parsing
- **Weaviate**: Multi-vector search

---

**Created**: October 2025
**For**: Cochrane RAG System
**Purpose**: Educational tutorial on multi-level chunking